# NLP Financials Metadata

This notebook will look to generate required NLP data of a financial return in the form that is accepted by the upstream program. I already have a Java program generating NLP data using Stanford CoreNLP, but I now realize that the quality of data generated in Python is better than what I currently have. It also provides more in-depth data and better control of the output. Therefore, it is pragmatic to take a hit right now and invest time in generating NLP data for catalog in Python. 

In [147]:
import nltk
import pickle
import gensim
import pandas as pd
import requests
import string
import datetime
import pattern
import hashlib
import numpy as np


from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
from nltk import word_tokenize, sent_tokenize
from nltk.stem.snowball import SnowballStemmer
from gensim.models.word2vec import Word2Vec
from gensim.models.phrases import Phraser, Phrases
from gensim import corpora
from sklearn.manifold import TSNE
from bs4 import BeautifulSoup
from gensim.utils import lemmatize

%matplotlib inline

ModuleNotFoundError: No module named 'tflearn'

In [132]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pshar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pshar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\pshar\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pshar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Each word CSV file must have: Symbol name, filing type, word, lemma, POS, NER, fileName, s#, and absolute path
Each sentence CSV file must have: Symbol name, filing type, fileName, s#, filePath (local or url), and actual sentence



Next, define a function that reads the input HTML file and filters out text devoid of any HTML text. 

In [133]:
def retrieveTxt(htmlPage):
  soup = BeautifulSoup(htmlPage, "lxml")
  tagTypes = ['div', 'p']
  tags = (soup).find_all(tagTypes)
  origTxt = ''
  for t in tags:
    origTxt += (t.text+" ")
  return origTxt

Clean the raw non-HTML text taking out stop words, punctuation marks, some special UTF-8 characters etc... Note that the cleanup here is result of earlier testing that has given me the best possible results thus far. 

In [134]:
def genIntermediateTokens(origTxt):
  # Start with taking out a UTF-8 token that seems to be prevalent
  intermediateTxt = origTxt.replace(u'\xa0', u' ')
  intermediateTokens = nltk.word_tokenize(intermediateTxt)
  return intermediateTokens

The next function takes care of the stop words. But the value of rest of the code in function is somewhat arguable. There are repeated for loops here to avoid one specific condition where words at the end of sentence is getting merged with word at the start of next sentence. TODO:  Look more into the problem, and avoid this double for loop. 
  

In [135]:
def cleanedWordsTxt(intermediateTokens):
  stopWords = set(stopwords.words('english') + list(string.punctuation))
  cleanTokens = []
  for w in intermediateTokens:
      if w not in stopWords:
          cleanTokens.append(w.lower())
          cleanTokens.append(' ') # Need to append a single space for cases where words are losing space in between
          
  cleanedTxt = ''
  cleanedTxtLst = []
  for token in cleanTokens:
    if token != ' ':
      cleanedTxtLst.append(token)
      cleanedTxt += (token)
  cleanedTxt = cleanedTxt.replace('  ', ' ')  
  return cleanedTxtLst

Thus far we have tokenized text into words and cleaned them. Now, do the same but keep the unit at sentence level rather than at word level. 

In [136]:
from nltk.tokenize import PunktSentenceTokenizer
def cleanedSentsTxt(origTxt):
  intermediateTxt = origTxt.replace(u'\xa0', u' ')
  sents_tokenized = sent_tokenize(intermediateTxt)
  sents_ClnTknzd = []
  punctuations = list(string.punctuation) # only remove punctuations. Keep stop words for phrases and un-abbreviated forms. Ex: United States "of" America. 

  for sent in sents_tokenized:
    tempStr = ''
    tempTokens = nltk.word_tokenize(sent)
    for token in tempTokens:
      if token not in punctuations:
          tempStr += (token)
          tempStr += ' '
          #cleanTokens.append(' ') # Need to append a single space for cases where words are losing space in between

    sents_ClnTknzd.append(tempStr.strip()) # Helps remove the space at the end of tempStr

  return sents_ClnTknzd


Adding a function to generate metadata such as word itself, lemma, NER, PoS etc... for individual words in a sentence. Earlier, sentences and words in those sentences were being pushed to two separate files for each symbol and filing. However, that was causing the unique ID to be different upon indexing for a sentence and words within it. That, in turn, would not allow sentences and words to both appear under a single result and there was no way to then group them. To avoid all that, the words metadata must get indexed with the sentence that includes those words in a single unique identifier within Solr. It presents a challenge in that while a sentence is unique with in a single row, now the "word" or "pos" or "ner" will no longer be unique even within a single document as Solr defines it. Each row will contain all the words and their respective pos within a single sentence. This will not only increase the size of input file for indexing, but will also increase duration and complexity. I spent few days thinking about alternate solutions but couldn't figure out a way to combine sentence and words annotation metadata if they were indexed at separate times, therefore, this effort now as an option of last resort. 

In [137]:
enableNER = False # Keep it disabled unless Stanford CoreNLP server is running locally and you are on a loaded (CPU, mem) instance
stopWords = set(stopwords.words('english') + list(string.punctuation))
from nltk.stem.wordnet import WordNetLemmatizer
# create an object of stemming function
stemmer = SnowballStemmer("english")
# lemmatizer = WordNetLemmatizer()

if (enableNER):
  from nltk.parse import CoreNLPParser
  try: # Careful when enabling NER generation. It can increase compute times by order of 20-30 times 
    ner_tagger = CoreNLPParser(url='http://localhost:9000', tagtype='ner')
  except:
    print("NER_Tagger not available. Ensure that local CoreNLP Server is running.")

def genLemmas(sent):
    i = 0
    tokens = word_tokenize(sent)
    pos = nltk.pos_tag(tokens)
    if (enableNER):
      ner = ner_tagger.tag(tokens)
    posLength = len(pos)
    # Higher level list which will contain map with token as "k" and list of lemma, ner, pos as "v"
    annListForSent = []
       
    try:
      for token in tokens:
        if token not in stopWords:
          tknEntry = {}
          lstTemp = [] 
          # lemma = lemmatizer.lemmatize(token) # Didn't work 
          lemma = stemmer.stem(token)
          posWord = pos[i][1] 
          if (enableNER): nerWord = ner[i][1] 
          else: nerWord = 'ner'

          lstTemp.append(lemma); lstTemp.append(posWord); lstTemp.append(nerWord) # 0:lemma, 1:pos, 2:ner, 
          tknEntry[token] = lstTemp


          annListForSent.append(tknEntry)
        i += 1 # Must increase value of i regardless of whether the word is in stopWords or not. Recall that POS and NER were computed with stopwords included. 
        
    except IndexError:
      print("List index out of range likely for PoS or NER tagger")
      print("Sentence where the error occured is: '\"+sent+\"'")
      print("Skipping indexing of this sentence.")
    
    return annListForSent
        

In [138]:
"""
sent = "The Company has entered into a guarantee for the benefit of PSA Group and pursuant to \
    which the Company has agreed to guarantee the Seller's obligation to indemnify PSA Group for certain \
    losses resulting from any inaccuracy of certain representations and warranties or breaches of our covenants \
    in the Agreement and for certain other liabilities."
annListForSent = genLemmas(sent)
annListForSent
"""

'\nsent = "The Company has entered into a guarantee for the benefit of PSA Group and pursuant to     which the Company has agreed to guarantee the Seller\'s obligation to indemnify PSA Group for certain     losses resulting from any inaccuracy of certain representations and warranties or breaches of our covenants     in the Agreement and for certain other liabilities."\nannListForSent = genLemmas(sent)\nannListForSent\n'

It is important to use sentences to figure out Parts of Speech. If you determine PoS just based on list of words then the context is likely to be lost.

In [139]:
def genAnnotations(symbol, filingType, cleanedSentsList, nameInputFile, url):
  annList = []
  sentList = []

  sCount = 0
  for sent in cleanedSentsList:
    sCount += 1
    
    
    lstSent = []
    lstSent.append(symbol); lstSent.append(filingType); lstSent.append(nameInputFile)
    lstSent.append('s'+str(sCount)); lstSent.append(url); lstSent.append(sent.replace(",", ""))
    # This is purely to group search results by sentence. Solr won't allow grouping by large text fields, so hash it. 
    hash = hashlib.sha224(bytes(sent.replace(",", ""), encoding='utf-8')).hexdigest()
    lstSent.append(hash)
    # Now, generate annotations at individual word level
    annListForSent = genLemmas(sent)
    cntWords = len(annListForSent)
    lstSent.append(cntWords)
    for kvPair in annListForSent:
        for key, value in kvPair.items():
            lstSent.append(key)
            lstSent.append(value[0])
            lstSent.append(value[1])
            lstSent.append(value[2])

    #tempSentStr = symbol + "," + filingType + "," + nameInputFile + "," + str(sCount) + "," + url + "," + sent.replace(",", "")
    sentList.append(lstSent)
    
  return sentList

In [140]:
import csv
filesLocal = True
dirNLPOut = 'C:/Users/pshar/Dropbox/Programming/SampleTexts/NLP_Output'
"""
urls = []
urls.append("https://www.sec.gov/Archives/edgar/data/886982/000156459019008879/gs-424b2.htm")
# Manual settings for the test run. 
symbol = 'GS'
filingType = '10-K'
nameInputFile = 'd480167d10k.htm'
"""

page = ''
def traverseFilingFiles(symbol, filingType, filePath):
    (parentDir, nameInputFile) = os.path.split(filePath)
    origTxt = ''
    if (filesLocal == False):
        # The filePath here should be URL. 
        # TODO: Handle upstream function to send URL or localPath both
        htmlPage = requests.get(filePath).text # Retrieve text for one file at one point
        origTxt = retrieveTxt(htmlPage)
    else: 
        try:
            with open(filePath, 'r', encoding='utf8') as content_file:
                htmlPage = content_file.read()
                origTxt = retrieveTxt(htmlPage)
        except UnicodeDecodeError: 
            print ("UnicodeDecodeError reading HTML: '"+filePath+"' for symbol:"+symbol)
        
    """
    # Commenting out individual words based function calls, while keeping analysis at the sentence level
    intermediateTokens = genIntermediateTokens(origTxt)
    cleanedWordsList = cleanedWordsTxt(intermediateTokens)
    """
    cleanedSentsList = cleanedSentsTxt(origTxt)
    sentList = genAnnotations(symbol, filingType, cleanedSentsList, nameInputFile, filePath)
    #print(sentList)

    now = datetime.datetime.now()
    time = str(now)[:19].replace(":", "-").replace(" ", "_")

    """
    # Disabling Annotations (words) output for the time-being until Solr cataloging is resolved
    
    fileWords = 'Annotations_'+symbol+'_'+filingType+'_'+nameInputFile+'_'+time+'.csv'
    absPathFileWords = os.path.join(dirNLPOut, fileWords)
    print(fileWords)
    dfrmWords = pd.DataFrame(annList)
    dfrmWords.to_csv(absPathFileWords, sep =",", index = None, header=False, quoting=csv.QUOTE_NONE, escapechar="\\")
    """
    
    fileSents = 'Sentences_'+symbol+'_'+filingType+'_'+nameInputFile+'_'+time+'.csv'
    absPathFileSents = os.path.join(dirNLPOut, fileSents)
    print(fileSents)
    dfrmSents = pd.DataFrame(sentList)#, columns = labelSents)
    #print(dfrmSents)
    dfrmSents.to_csv(absPathFileSents, sep =",", index = None, header=False, quoting=csv.QUOTE_NONE, escapechar="\\")    

# Retrieve & Traverse Filings Previously Downloaded 

### Base functions to traverse symbols, then filing types (10-k, 10-Q...), and then individual filings

Start with listing symbols whose filings were earlier downloaded in a dir listing arranged by symbols

In [141]:
import os
from os.path import isdir
from os.path import abspath
"""
The base location locally 
"""
def genListDirsSymbols(basePath):
    symbols = []
    dirsSymbols = os.listdir(basePath)
    for dirSymbol in dirsSymbols:
        (head, tail) = os.path.split(dirSymbol)
        symbols.append(tail)
    return symbols

For each symbol, generate listing of folders underneath. This listing will correspond to filing types (10-k etc...) downloaded for each of these symbols. 

In [142]:

def genListFilingsDirsForSymbol(basePathPlusSymbol):
    filingDirs = []
    dirsFilings = os.listdir(basePathPlusSymbol)
    for dirFilings in dirsFilings:
        (head, tail) = os.path.split(dirFilings)
        filingDirs.append(tail)
    return filingDirs   

Retrieve listing of files for given symbol and given filing type.

In [143]:
def genListFilingsForSymbol(basePathPlusSymbolPlusFilingDir):
    filings = []
    files = os.listdir(basePathPlusSymbolPlusFilingDir)
    for file in files:
        (head, tail) = os.path.split(file)
        if (tail != 'txt'): 
            filings.append(file)
    return filings

    

Now, that we have functions defined to list symbols, filing types, and actual files defined, go through them as a list and traverse these files grouping them first by symbol and then by filing type.

In [146]:
dirBase = 'C:/Users/pshar/Dropbox/WebServices/mysite/polls/templates/polls/FilingsBySymbols'
filingsFullLst = []
symbols = genListDirsSymbols(dirBase)
for symbol in symbols:
    basePathPlusSymbol = os.path.join(dirBase, symbol.upper())
    filingDirs = genListFilingsDirsForSymbol(basePathPlusSymbol)
    for filingDir in filingDirs:
        basePathPlusSymbolPlusFilingDir =  os.path.join(basePathPlusSymbol, filingDir)
        filings = genListFilingsForSymbol(basePathPlusSymbolPlusFilingDir)
        for filing in filings:
            filePath = os.path.join(basePathPlusSymbolPlusFilingDir, filing)
            traverseFilingFiles(symbol, filingDir, filePath)
               
            
    
print("DONE!")

Sentences_BAC_10-K_bac1231201610xkhtm_2019-04-02_21-27-38.csv
Sentences_BAC_10-K_bac1231201710xkhtm_2019-04-02_21-30-09.csv
Sentences_BAC_10-K_bac1231201810xkhtm_2019-04-02_21-31-01.csv
Sentences_BAC_10-Q_bac331201810xqhtm_2019-04-02_21-31-42.csv
Sentences_BAC_10-Q_bac630201710xqhtm_2019-04-02_21-32-38.csv
Sentences_BAC_10-Q_bac630201810xqhtm_2019-04-02_21-33-28.csv
Sentences_BAC_10-Q_bac930201710xqhtm_2019-04-02_21-34-19.csv
Sentences_BAC_10-Q_bac930201810xqhtm_2019-04-02_21-35-08.csv
Sentences_C_10-K_c12312016x10khtm_2019-04-02_21-36-38.csv
Sentences_C_10-K_c12312017x10khtm_2019-04-02_21-40-24.csv
Sentences_C_10-K_c12312018x10khtm_2019-04-02_21-43-47.csv
Sentences_C_10-Q_c3312018x10qhtm_2019-04-02_21-46-30.csv
Sentences_C_10-Q_c6302017x10qhtm_2019-04-02_21-47-35.csv
Sentences_C_10-Q_c6302018x10qhtm_2019-04-02_21-49-08.csv
Sentences_C_10-Q_c9302017x10qhtm_2019-04-02_21-50-43.csv
Sentences_C_10-Q_c9302018x10qhtm_2019-04-02_21-52-14.csv
Sentences_GS_10-K_d308759d10khtm_2019-04-02_21-53-

## Possible Enhancements

- P1: The Lemmatization API being called is yielding poor results. Not relying on lemma right now but need to correct that later. 
- P1: Keep reference to NLP_Financials_Cloud.ipynb for other possible enhancements.
- P1: Can we introduce topic modeling for each paragraph? The p HTML tag is already defined.  
- P1: Bring complete cycle (download, generate NLP, and indexing) here rather than just the middle part. 
- P1: If not full cycle, then keep the ability to generate NLP from both local files as well as from EDGAR. Couple of function may require little tweaking. 
